<a href="https://colab.research.google.com/github/Rexoes/-dev-3/blob/main/AUE_PSO_ANN_Hyperparameter_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import random

In [2]:
# 1. Veri Setini Oluşturma ve Veri Önişleme
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, n_informative=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Numpy kullanarak sınıf dağılımını öğrenme
unique, counts = np.unique(y, return_counts=True)
class_distribution_np = dict(zip(unique, counts))
print("Sınıf Dağılımı (Numpy):", class_distribution_np)

print(X_train.shape[1])

Sınıf Dağılımı (Numpy): {0: 499, 1: 501}
20


In [3]:
# 2. Model Oluşturma Fonksiyonu (Ek hiperparametreler ve daha karmaşık yapı)
def create_ann_model(num_layers, num_neurons, learning_rate, dropout_rate, activation="relu"):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(shape=(X_train.shape[1],)))  # Giriş boyutunu doğrudan X_train.shape[1] olarak ayarladık
    for _ in range(num_layers):
        model.add(tf.keras.layers.Dense(num_neurons, activation=activation))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Sabit olarak Adam optimizer kullanılıyor
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# 3. PSO Parametreleri
num_particles = 10
num_iterations = 10

# Adaptif w, c1 ve c2 geniş aralıklarda başlangıç ve bitiş değerleri
w_start, w_end = 1.2, 0.2
c1_start, c1_end = 3.0, 0.4
c2_start, c2_end = 0.4, 3.0

# Genişletilmiş Hiperparametre sınırları ve değer kümeleri
hyperparam_bounds = {
    "num_layers": (1, 3),
    "num_neurons": (8, 64),
    "learning_rate": (1e-4, 1e-2),
    "dropout_rate": (0.1, 0.5),
    "batch_size": (32, 64),
    "activation": ["relu", "tanh"]
}

def random_hyperparameters():
    return {
        "num_layers": random.randint(hyperparam_bounds["num_layers"][0], hyperparam_bounds["num_layers"][1]),
        "num_neurons": random.randint(hyperparam_bounds["num_neurons"][0], hyperparam_bounds["num_neurons"][1]),
        "learning_rate": random.uniform(hyperparam_bounds["learning_rate"][0], hyperparam_bounds["learning_rate"][1]),
        "dropout_rate": random.uniform(hyperparam_bounds["dropout_rate"][0], hyperparam_bounds["dropout_rate"][1]),
        "batch_size": random.randint(hyperparam_bounds["batch_size"][0], hyperparam_bounds["batch_size"][1]),
        "activation": random.choice(hyperparam_bounds["activation"])
    }

In [5]:
# Parçacık sınıfı, her bir parçacık bir hiperparametre seti olarak ele alınır
class Particle:
    def __init__(self):
        self.position = random_hyperparameters()
        self.velocity = {key: 0 for key in self.position}
        self.best_position = self.position.copy()
        self.best_score = float('inf')

    def update_velocity(self, global_best_position, w, c1, c2):
        for key in self.position:
            if isinstance(self.position[key], (int, float)):
                r1, r2 = random.random(), random.random()
                cognitive_velocity = c1 * r1 * (self.best_position[key] - self.position[key])
                social_velocity = c2 * r2 * (global_best_position[key] - self.position[key])
                self.velocity[key] = w * self.velocity[key] + cognitive_velocity + social_velocity

    def update_position(self):
        for key in self.position:
            if isinstance(self.position[key], int):
                self.position[key] = int(self.position[key] + self.velocity[key])
                # Sınırları kontrol et ve sınır dışı değerleri ayarlanıyor
                self.position[key] = max(hyperparam_bounds[key][0], min(hyperparam_bounds[key][1], self.position[key]))
            elif isinstance(self.position[key], float):
                self.position[key] += self.velocity[key]
                # Sınırları kontrol et ve sınır dışı değerleri ayarlanıyor
                self.position[key] = max(hyperparam_bounds[key][0], min(hyperparam_bounds[key][1], self.position[key]))

In [6]:
# TensorFlow GPU desteğini kontrol edelim (Colab veya yerel makine)
print("GPU available:", tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU available: True


In [7]:
# 4. PSO Algoritması
particles = [Particle() for _ in range(num_particles)]
global_best_position = particles[0].position
global_best_score = float('inf')

for iteration in range(num_iterations):
    # Adaptif w, c1 ve c2 değerleri
    w = w_start - (iteration / num_iterations) * (w_start - w_end)
    c1 = c1_start - (iteration / num_iterations) * (c1_start - c1_end)
    c2 = c2_start + (iteration / num_iterations) * (c2_end - c2_start)

    for particle in particles:
        # Parçacığın hiperparametre değerleriyle modelin oluşturulması
        model = create_ann_model(
            particle.position["num_layers"],
            particle.position["num_neurons"],
            particle.position["learning_rate"],
            particle.position["dropout_rate"],
            activation=particle.position["activation"]
        )

        # Modeli parçacığın belirlediği batch_size ile eğitilmesi
        batch_size = particle.position["batch_size"]
        history = model.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=0)
        y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
        accuracy = accuracy_score(y_test, y_pred)
        score = 1 - accuracy  # Düşük skor daha iyi

        # Parçacıkların kişisel ve global en iyi pozisyonlarının güncellenmesi
        if score < particle.best_score:
            particle.best_score = score
            particle.best_position = particle.position.copy()
        if score < global_best_score:
            global_best_score = score
            global_best_position = particle.position.copy()

    # Hız ve pozisyon güncelleme
    for particle in particles:
        particle.update_velocity(global_best_position, w, c1, c2)
        particle.update_position()

    print(f"Iteration {iteration + 1}/{num_iterations}, Best Accuracy: {1 - global_best_score}, w: {w}, c1: {c1}, c2: {c2}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
Iteration 1/10, Best Accuracy: 0.8566666666666667, w: 1.2, c1: 3.0, c2: 0.4
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Iteration 2/10, Best Accuracy: 0.8566666666666667, w: 1.0999999999999999, c1: 2.74, c2: 0.66
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1

In [8]:
# 5. En İyi Hiperparametre Seti ile Nihai Model
print("Optimal Hiperparametreler:", global_best_position)
best_model = create_ann_model(
    global_best_position["num_layers"],
    global_best_position["num_neurons"],
    global_best_position["learning_rate"],
    global_best_position["dropout_rate"],
    activation=global_best_position["activation"]
)
best_batch_size = global_best_position["batch_size"]
history = best_model.fit(X_train, y_train, epochs=50, batch_size=best_batch_size, validation_data=(X_test, y_test))

Optimal Hiperparametreler: {'num_layers': 1, 'num_neurons': 53, 'learning_rate': 0.007550675895783749, 'dropout_rate': 0.3323400598115418, 'batch_size': 35, 'activation': 'relu'}
Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.6071 - loss: 0.7767 - val_accuracy: 0.8633 - val_loss: 0.3843
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8593 - loss: 0.3471 - val_accuracy: 0.8967 - val_loss: 0.3090
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8991 - loss: 0.2403 - val_accuracy: 0.9033 - val_loss: 0.2753
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9032 - loss: 0.2537 - val_accuracy: 0.9167 - val_loss: 0.2546
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9315 - loss: 0.1701 - val_accuracy: 0.9233 - val_loss: 0.2225
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9238 - loss: 0.1867 - val_accuracy: 0.9200 - val_loss: 0.2133
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.